In [9]:
!pip install beautifulsoup4
!pip install selenium

You should consider upgrading via the 'C:\Users\leedo\miniconda3\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\leedo\miniconda3\python.exe -m pip install --upgrade pip' command.


In [1]:
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
import time
from tqdm import tqdm_notebook
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
import re
import urllib.request
import os
import csv

In [2]:
url = "https://www.mangoplate.com/"
driver = webdriver.Chrome("./driver/chromedriver.exe")
driver.get(url)
driver.maximize_window()

C:\Users\leedo\AppData\Local\Temp\ipykernel_26864\3248022642.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("./driver/chromedriver.exe")


In [3]:
# 검색창에 서울지역 검색
search_box = driver.find_element_by_xpath('//*[@id="main-search"]')
search_box.send_keys('서울')
search_box.send_keys(Keys.RETURN)

C:\Users\leedo\AppData\Local\Temp\ipykernel_26864\615670643.py:2: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  search_box = driver.find_element_by_xpath('//*[@id="main-search"]')


In [4]:
# 필터 선택
driver.find_element_by_xpath('/html/body/main/article/div[2]/div/div/section/div[1]/div[1]/button[1]').click()
time.sleep(1)
# 지역 선택 서울-강남: 1, 서울-강북: 2
driver.find_element_by_xpath('/html/body/main/article/div[2]/div/aside/div[1]/div/div[3]/p[1]/a[{}]'.format(2)).click()

# 세부지역 선택 강남: 1~26, 강북: 1~25
driver.find_element_by_xpath('/html/body/main/article/div[2]/div/aside/div[1]/div/div[3]/p[2]/span[{}]/label/span'.format(1)).click()

# 지역 변수 저장 강남: 1~26, 강북: 1~25
html = driver.page_source
soup = BeautifulSoup(html, "html.parser")
district = soup.select("body > main > article > div.column-wrapper > div > aside > div.inner > div > div:nth-child(3) > p.metro > span:nth-child({}) > label > span".format(1))[0].text
district = district.replace("/", "_") + '_'

# 가격 만원미만, 1만원대, 2만원대, 3만원대 선택: 1 ~ 4
driver.find_element_by_xpath('/html/body/main/article/div[2]/div/aside/div[1]/div/div[2]/p/label[{}]'.format(1)).click()

# 음식종류 선택: 한식, 일식, 중식, 양식, 세계음식, 뷔페, 카페, 주점: 1~8
driver.find_element_by_xpath('/html/body/main/article/div[2]/div/aside/div[1]/div/div[4]/p/label[{}]'.format(1)).click()
time.sleep(1)

# 필터창 선택버튼
driver.find_element_by_xpath('/html/body/main/article/div[2]/div/aside/div[2]/button[2]').click()
time.sleep(1)
html = driver.page_source
soup = BeautifulSoup(html, "html.parser")

C:\Users\leedo\AppData\Local\Temp\ipykernel_26864\3131256044.py:2: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('/html/body/main/article/div[2]/div/div/section/div[1]/div[1]/button[1]').click()
C:\Users\leedo\AppData\Local\Temp\ipykernel_26864\3131256044.py:5: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('/html/body/main/article/div[2]/div/aside/div[1]/div/div[3]/p[1]/a[{}]'.format(2)).click()
C:\Users\leedo\AppData\Local\Temp\ipykernel_26864\3131256044.py:8: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('/html/body/main/article/div[2]/div/aside/div[1]/div/div[3]/p[2]/span[{}]/label/span'.format(1)).click()
C:\Users\leedo\AppData\Local\Temp\ipykernel_26864\3131256044.py:17: DeprecationWarn

In [5]:
# 페이지 내 20개 식당 클릭 후 크롤링
def page_one_crawling():
    for q in range(1,11):
            for w in range(1,3):
                target = driver.find_element_by_xpath("/html/body/main/article/div[2]/div/div/section/div[3]/ul/li[{}]/div[{}]/figure/figcaption/div/a".format(q,w))
                target.send_keys(Keys.CONTROL +"\n")
                time.sleep(0.3)
                driver.switch_to.window(driver.window_handles[-1])
                html = driver.page_source
                soup = BeautifulSoup(html, "html.parser")

                # 리뷰 개수
                review_count_out = soup.select("body > main > article > div.column-wrapper > div.column-contents > div > section.RestaurantReviewList > header > h2 > span.RestaurantReviewList__AllCount")[0].text

                # 평점없으면 None
                try:
                    point = soup.select("body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > header > div.restaurant_title_wrap > span > strong > span")[0].text
                except:
                    point = None


                if int(review_count_out) >= 1:

                    # 더보기 칸까지 스크롤 내리기
                    try:
                        driver.find_element_by_xpath('/html/body/main/article/div[1]/div[1]/div/section[3]/div[2]')
                        action = ActionChains(driver)
                        action.move_to_element(driver.find_element_by_xpath('/html/body/main/article/div[1]/div[1]/div/section[3]/div[2]')).perform()
                    except:
                        pass
                    try:
                        for e in range(int(review_count_out)//5):
                            driver.find_element_by_xpath('/html/body/main/article/div[1]/div[1]/div/section[3]/div[2]').click()
                            time.sleep(0.3)

                            action = ActionChains(driver)
                            action.move_to_element(driver.find_element_by_xpath('/html/body/main/article/div[1]/div[1]/div/section[3]/div[2]')).perform()
                    except:
                        pass
                    html = driver.page_source
                    soup = BeautifulSoup(html, "html.parser")

                    # 스크롤 내려서 삭제된 리뷰 제외된 실제 개수
                    review_count = len(soup.select("body > main > article > div.column-wrapper > div.column-contents > div > section.RestaurantReviewList > ul")[0]) -1

                    # 실제 리뷰개수가 1개부터 조사
                    if review_count >= 1:
                        # 가게 이름
                        name = soup.select("body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > header > div.restaurant_title_wrap > span > h1")[0].text
                        save_path = "/Users/leedo/Documents/ds_study/" + district + name + "/"

                        # 지역 + 가게 이름으로 폴더만들기
    #                     try:
    #                         if not os.path.isdir(save_path):
    #                             os.mkdir(save_path)
    #                     except:
    #                         pass

                        driver.switch_to.window(driver.window_handles[-1])
                        html = driver.page_source
                        soup = BeautifulSoup(html, "html.parser")

                        detail_count = soup.select("body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > table > tbody > tr > th")

                        for i in range(len(detail_count)):
                            if detail_count[i].text == '주소':
                                address = soup.select("body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > table > tbody > tr:nth-child({})".format(i + 1))[0].text

                                address = re.sub('\n', ' ', address).strip().split(' ')

                                for i in range(len(address)):
                                    if address[i] == '지번':
                                        a=address[1:i]
                                address = ' '.join(a)


                            elif detail_count[i].text == '음식 종류':

                                category = soup.select("body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > table > tbody > tr:nth-child({})".format(i + 1))[0].text
                                category = re.sub('\n', ' ', category).strip().split(' ')
                                category = category[3:]
                                category = ' '.join(category)

                            elif detail_count[i].text == '가격대':

                                price = soup.select("body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > table > tbody > tr:nth-child({})".format(i + 1))[0].text
                                price = re.sub('\n', ' ', price).strip().split(' ')
                                price = price[1:]
                                price = ' '.join(price)


                        # 스크롤 내려서 삭제된 리뷰 제외된 실제 개수
                        review_count = int(len(soup.select("body > main > article > div.column-wrapper > div.column-contents > div > section.RestaurantReviewList > ul")[0]) -1)

                        # 리뷰 크롤링
                        try:
                            for i in range(1,int(review_count)+1):
                                tag = "body > main > article > div.column-wrapper > div.column-contents > div > section.RestaurantReviewList > ul > li:nth-child({}) > a > div.RestaurantReviewItem__ReviewContent > div > p".format(i)
                                review = soup.select(tag)
                                review = review[0].text
                                review = str(re.sub('\n', ' ', review).strip())

                                # 맛있다 또는 괜찮다 표시
                                recommend = driver.find_element_by_xpath("/html/body/main/article/div[1]/div[1]/div/section[3]/ul/li[{}]/a/div[3]/span".format(i)).text
                                if review != '':
                                    review_list.append({'가게이름':name, '리뷰':review, '맛평가': recommend})
                        except:
                            pass

                        detail.append({'가게이름':name,'가게 주소':address,'음식 종류':category,'가격 범위':price,"리뷰개수":review_count,'평점':point})

                        # 리뷰 펼친 후 스크롤이 해당 지점에 못 미쳐서 클릭 조회가 안되던 걸 리뷰 개수 위치로 스크롤 이동 후 해결
                        action.move_to_element(driver.find_element_by_xpath('/html/body/main/article/div[1]/div[1]/div/section[3]/header/h2/span[4]')).perform()
                        # n 번째 리뷰 클릭후 썸네일 다운
                        count = 0

                        try:               
    #                         for r in range(review_count):
    #                             tag = '/html/body/main/article/div[1]/div[1]/div/section[3]/ul/li[{}]/a/div[2]/div/p'.format(r+1)
    #                             driver.find_element_by_xpath(tag).click()

    #                             action = ActionChains(driver)
    #                             action.move_to_element(driver.find_element_by_xpath(tag)).perform()

    #                             time.sleep(0.5)

    #                             driver.switch_to.window(driver.window_handles[-1])
    #                             html = driver.page_source
    #                             soup = BeautifulSoup(html, "html.parser")

    #                             # 썸네일 갯수
    #                             html = driver.page_source
    #                             soup = BeautifulSoup(html, "html.parser")
    #                             if len(soup.select("body > main > section > div > div")) != 0:
    #                                 picture_count = int((len(soup.select("body > main > section > div > div")[0]) - 1) / 2)

    #                                 # n 번째 썸네일 url확인 후 저장
    #                                 for t in range(picture_count):
    #                                     count += 1
    #                                     try:
    #                                         image_url = driver.find_element_by_xpath("/html/body/main/section/div/div/div[{}]".format(t+1)).get_attribute("style")
    #                                         a = image_url.index('(')
    #                                         b = image_url.index(')')
    #                                         image_url = image_url[a+2:b-1]
    #                                         urllib.request.urlretrieve(image_url, save_path +'{}.jpg'.format(count))
    #                                         time.sleep(0.1)
    #                                     except:
    #                                         pass
    #                                 driver.close()  #링크 이동 후 탭 닫기
    #                                 driver.switch_to.window(driver.window_handles[-1])
    #                                 time.sleep(0.1)
    #                             else:
    #                                 driver.close()  #링크 이동 후 탭 닫기
    #                                 driver.switch_to.window(driver.window_handles[-1])
    #                                 time.sleep(0.1)

                            driver.close()  #링크 이동 후 탭 닫기
                            driver.switch_to.window(driver.window_handles[-1])
                            time.sleep(0.1)
                        except:
                            driver.close()  #링크 이동 후 탭 닫기
                            driver.switch_to.window(driver.window_handles[-1])
                            time.sleep(0.1)

                    else:            
                        driver.close()  #링크 이동 후 탭 닫기
                        driver.switch_to.window(driver.window_handles[-1])
                        time.sleep(0.1)
                else:
                    driver.close()  #링크 이동 후 탭 닫기
                    driver.switch_to.window(driver.window_handles[-1])
                    time.sleep(0.1)

In [6]:
def page_start_to_end_crawling():
    for z in range(2,11):
        try:
            page_one_crawling()
            driver.find_element_by_xpath('/html/body/main/article/div[2]/div/div/section/div[4]/p/a[{}]'.format(z)).click()
            time.sleep(1)
            if z == 10:
                page_one_crawling()
                time.sleep(1)
        except:
            break

In [7]:
def filter_price_button_option():
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")

    # 필터선택 버튼 스크롤
    action = ActionChains(driver)
    action.move_to_element(driver.find_element_by_xpath('/html/body/main/article/div[2]/div/div/section/div[1]/div[1]/button[1]')).perform()
    time.sleep(0.3)
    
    # 필터 선택
    driver.find_element_by_xpath('/html/body/main/article/div[2]/div/div/section/div[1]/div[1]/button[1]').click()
    time.sleep(0.3)

    # 가격선택 버튼 스크롤
    action = ActionChains(driver)
    action.move_to_element(driver.find_element_by_xpath('/html/body/main/article/div[2]/div/aside/div[1]/div/div[2]/p/label[1]')).perform()
    time.sleep(0.3)

    # 가격 만원미만, 1만원대, 2만원대, 3만원대 선택: 1 ~ 4
    driver.find_element_by_xpath('/html/body/main/article/div[2]/div/aside/div[1]/div/div[2]/p/label[{}]'.format(x)).click()
    driver.find_element_by_xpath('/html/body/main/article/div[2]/div/aside/div[1]/div/div[2]/p/label[{}]'.format(x+1)).click()
    time.sleep(0.3)

    # 필터창 선택버튼
    driver.find_element_by_xpath('/html/body/main/article/div[2]/div/aside/div[2]/button[2]').click()
    time.sleep(0.3)
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")

In [8]:
def filter_category_button_option():
    # 필터선택 버튼 스크롤
    action = ActionChains(driver)
    action.move_to_element(driver.find_element_by_xpath('/html/body/main/article/div[2]/div/div/section/div[1]/div[1]/button[1]')).perform()    
    time.sleep(0.3)
    
    # 필터 선택
    driver.find_element_by_xpath('/html/body/main/article/div[2]/div/div/section/div[1]/div[1]/button[1]').click()
    time.sleep(0.3)

    driver.find_element_by_xpath('/html/body/main/article/div[2]/div/aside/div[1]/div/div[2]/p/label[1]').click()
    driver.find_element_by_xpath('/html/body/main/article/div[2]/div/aside/div[1]/div/div[2]/p/label[{}]'.format(x+1)).click()

    # 음식종류선택 버튼 스크롤
    action = ActionChains(driver)
    action.move_to_element(driver.find_element_by_xpath('/html/body/main/article/div[2]/div/aside/div[1]/div/div[4]/p/label[5]')).perform()
    time.sleep(0.3)
    
    # 음식종류 선택: 한식, 일식, 중식, 양식, 세계음식, 뷔페, 카페, 주점: 1~8
    driver.find_element_by_xpath('/html/body/main/article/div[2]/div/aside/div[1]/div/div[4]/p/label[{}]'.format(s)).click()
    driver.find_element_by_xpath('/html/body/main/article/div[2]/div/aside/div[1]/div/div[4]/p/label[{}]'.format(s+1)).click()
    time.sleep(0.3)

    # 필터창 선택버튼
    driver.find_element_by_xpath('/html/body/main/article/div[2]/div/aside/div[2]/button[2]').click()
    time.sleep(0.3)
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")

In [9]:
detail = []
review_list = []

In [10]:
# total
# 리뷰만 크롤링
# 1 - 10 페이지, 2부터 시작
for s in range(1,9):
    try:
        for x in range(1,4):
            page_start_to_end_crawling()
            filter_price_button_option()
            
            if x == 3:
                page_start_to_end_crawling()
        filter_category_button_option()
    except:
        pass

C:\Users\leedo\AppData\Local\Temp\ipykernel_26864\1049950786.py:5: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  target = driver.find_element_by_xpath("/html/body/main/article/div[2]/div/div/section/div[3]/ul/li[{}]/div[{}]/figure/figcaption/div/a".format(q,w))
C:\Users\leedo\AppData\Local\Temp\ipykernel_26864\1049950786.py:26: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('/html/body/main/article/div[1]/div[1]/div/section[3]/div[2]')
C:\Users\leedo\AppData\Local\Temp\ipykernel_26864\1049950786.py:28: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  action.move_to_element(driver.find_element_by_xpath('/html/body/main/article/div[1]/div[1]/div/section[3]/div[2]')).perform()
C:\Users\leedo\AppData\Local\Temp\ipykernel_26864\1049950786.py:33: Deprecation

In [56]:
# 리뷰만 크롤링
# 1 - 10 페이지, 2부터 시작
for z in range(2,11):
#     for q in range(1,11):
#         for w in range(1,3):
#             target = driver.find_element_by_xpath("/html/body/main/article/div[2]/div/div/section/div[3]/ul/li[{}]/div[{}]/figure/figcaption/div/a".format(q,w))
#             target.send_keys(Keys.CONTROL +"\n")
#             time.sleep(0.3)
#             driver.switch_to.window(driver.window_handles[-1])
#             html = driver.page_source
#             soup = BeautifulSoup(html, "html.parser")

#             # 리뷰 개수
#             review_count_out = soup.select("body > main > article > div.column-wrapper > div.column-contents > div > section.RestaurantReviewList > header > h2 > span.RestaurantReviewList__AllCount")[0].text

#             # 평점없으면 None
#             try:
#                 point = soup.select("body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > header > div.restaurant_title_wrap > span > strong > span")[0].text
#             except:
#                 point = None


#             if int(review_count_out) >= 1:

#                 # 더보기 칸까지 스크롤 내리기
#                 try:
#                     driver.find_element_by_xpath('/html/body/main/article/div[1]/div[1]/div/section[3]/div[2]')
#                     action = ActionChains(driver)
#                     action.move_to_element(driver.find_element_by_xpath('/html/body/main/article/div[1]/div[1]/div/section[3]/div[2]')).perform()
#                 except:
#                     pass
#                 try:
#                     for e in range(int(review_count_out)//5):
#                         driver.find_element_by_xpath('/html/body/main/article/div[1]/div[1]/div/section[3]/div[2]').click()
#                         time.sleep(0.3)

#                         action = ActionChains(driver)
#                         action.move_to_element(driver.find_element_by_xpath('/html/body/main/article/div[1]/div[1]/div/section[3]/div[2]')).perform()
#                 except:
#                     pass
#                 html = driver.page_source
#                 soup = BeautifulSoup(html, "html.parser")

#                 # 스크롤 내려서 삭제된 리뷰 제외된 실제 개수
#                 review_count = len(soup.select("body > main > article > div.column-wrapper > div.column-contents > div > section.RestaurantReviewList > ul")[0]) -1

#                 # 실제 리뷰개수가 1개부터 조사
#                 if review_count >= 1:
#                     # 가게 이름
#                     name = soup.select("body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > header > div.restaurant_title_wrap > span > h1")[0].text
#                     save_path = "/Users/leedo/Documents/ds_study/" + district + name + "/"

#                     # 지역 + 가게 이름으로 폴더만들기
# #                     try:
# #                         if not os.path.isdir(save_path):
# #                             os.mkdir(save_path)
# #                     except:
# #                         pass

#                     driver.switch_to.window(driver.window_handles[-1])
#                     html = driver.page_source
#                     soup = BeautifulSoup(html, "html.parser")

#                     detail_count = soup.select("body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > table > tbody > tr > th")

#                     for i in range(len(detail_count)):
#                         if detail_count[i].text == '주소':
#                             address = soup.select("body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > table > tbody > tr:nth-child({})".format(i + 1))[0].text

#                             address = re.sub('\n', ' ', address).strip().split(' ')

#                             for i in range(len(address)):
#                                 if address[i] == '지번':
#                                     a=address[1:i]
#                             address = ' '.join(a)


#                         elif detail_count[i].text == '음식 종류':

#                             category = soup.select("body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > table > tbody > tr:nth-child({})".format(i + 1))[0].text
#                             category = re.sub('\n', ' ', category).strip().split(' ')
#                             category = category[3:]
#                             category = ' '.join(category)

#                         elif detail_count[i].text == '가격대':

#                             price = soup.select("body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > table > tbody > tr:nth-child({})".format(i + 1))[0].text
#                             price = re.sub('\n', ' ', price).strip().split(' ')
#                             price = price[1:]
#                             price = ' '.join(price)


#                     # 스크롤 내려서 삭제된 리뷰 제외된 실제 개수
#                     review_count = int(len(soup.select("body > main > article > div.column-wrapper > div.column-contents > div > section.RestaurantReviewList > ul")[0]) -1)

#                     # 리뷰 크롤링
#                     try:
#                         for i in range(1,int(review_count)+1):
#                             tag = "body > main > article > div.column-wrapper > div.column-contents > div > section.RestaurantReviewList > ul > li:nth-child({}) > a > div.RestaurantReviewItem__ReviewContent > div > p".format(i)
#                             review = soup.select(tag)
#                             review = review[0].text
#                             review = str(re.sub('\n', ' ', review).strip())
                            
#                             # 맛있다 또는 괜찮다 표시
#                             recommend = driver.find_element_by_xpath("/html/body/main/article/div[1]/div[1]/div/section[3]/ul/li[{}]/a/div[3]/span".format(i)).text
#                             if review != '':
#                                 review_list.append({'가게이름':name, '리뷰':review, '맛평가': recommend})
#                     except:
#                         pass

#                     detail.append({'가게이름':name,'가게 주소':address,'음식 종류':category,'가격 범위':price,"리뷰개수":review_count,'평점':point})

#                     # 리뷰 펼친 후 스크롤이 해당 지점에 못 미쳐서 클릭 조회가 안되던 걸 리뷰 개수 위치로 스크롤 이동 후 해결
#                     action.move_to_element(driver.find_element_by_xpath('/html/body/main/article/div[1]/div[1]/div/section[3]/header/h2/span[4]')).perform()
#                     # n 번째 리뷰 클릭후 썸네일 다운
#                     count = 0

#                     try:               
# #                         for r in range(review_count):
# #                             tag = '/html/body/main/article/div[1]/div[1]/div/section[3]/ul/li[{}]/a/div[2]/div/p'.format(r+1)
# #                             driver.find_element_by_xpath(tag).click()

# #                             action = ActionChains(driver)
# #                             action.move_to_element(driver.find_element_by_xpath(tag)).perform()

# #                             time.sleep(0.5)

# #                             driver.switch_to.window(driver.window_handles[-1])
# #                             html = driver.page_source
# #                             soup = BeautifulSoup(html, "html.parser")

# #                             # 썸네일 갯수
# #                             html = driver.page_source
# #                             soup = BeautifulSoup(html, "html.parser")
# #                             if len(soup.select("body > main > section > div > div")) != 0:
# #                                 picture_count = int((len(soup.select("body > main > section > div > div")[0]) - 1) / 2)

# #                                 # n 번째 썸네일 url확인 후 저장
# #                                 for t in range(picture_count):
# #                                     count += 1
# #                                     try:
# #                                         image_url = driver.find_element_by_xpath("/html/body/main/section/div/div/div[{}]".format(t+1)).get_attribute("style")
# #                                         a = image_url.index('(')
# #                                         b = image_url.index(')')
# #                                         image_url = image_url[a+2:b-1]
# #                                         urllib.request.urlretrieve(image_url, save_path +'{}.jpg'.format(count))
# #                                         time.sleep(0.1)
# #                                     except:
# #                                         pass
# #                                 driver.close()  #링크 이동 후 탭 닫기
# #                                 driver.switch_to.window(driver.window_handles[-1])
# #                                 time.sleep(0.1)
# #                             else:
# #                                 driver.close()  #링크 이동 후 탭 닫기
# #                                 driver.switch_to.window(driver.window_handles[-1])
# #                                 time.sleep(0.1)

#                         driver.close()  #링크 이동 후 탭 닫기
#                         driver.switch_to.window(driver.window_handles[-1])
#                         time.sleep(0.1)
#                     except:
#                         driver.close()  #링크 이동 후 탭 닫기
#                         driver.switch_to.window(driver.window_handles[-1])
#                         time.sleep(0.1)

#                 else:            
#                     driver.close()  #링크 이동 후 탭 닫기
#                     driver.switch_to.window(driver.window_handles[-1])
#                     time.sleep(0.1)
#             else:
#                 driver.close()  #링크 이동 후 탭 닫기
#                 driver.switch_to.window(driver.window_handles[-1])
#                 time.sleep(0.1)
    driver.find_element_by_xpath('/html/body/main/article/div[2]/div/div/section/div[4]/p/a[{}]'.format(z)).click()
    time.sleep(1)

C:\Users\leedo\AppData\Local\Temp\ipykernel_8528\2562595837.py:174: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('/html/body/main/article/div[2]/div/div/section/div[4]/p/a[{}]'.format(z)).click()


In [11]:
detail

[{'가게이름': '원조할아버지손두부',
  '가게 주소': '서울특별시 광진구 자양로 324',
  '음식 종류': '한정식 / 백반 / 정통 한식',
  '가격 범위': '만원 미만',
  '리뷰개수': 72,
  '평점': '4.5'},
 {'가게이름': '이화만두',
  '가게 주소': '서울특별시 광진구 능동로 278-1',
  '음식 종류': '한정식 / 백반 / 정통 한식',
  '가격 범위': '만원 미만',
  '리뷰개수': 6,
  '평점': '4.3'},
 {'가게이름': '서북면옥',
  '가게 주소': '서울특별시 광진구 자양로 199-1',
  '음식 종류': '국수 / 면 요리',
  '가격 범위': '만원 미만',
  '리뷰개수': 40,
  '평점': '4.2'},
 {'가게이름': '궤도에오르다',
  '가게 주소': '서울특별시 광진구 광나루로 416 건희빌딩 1F',
  '음식 종류': '기타 한식',
  '가격 범위': '만원 미만',
  '리뷰개수': 40,
  '평점': '4.0'},
 {'가게이름': '콩이네',
  '가게 주소': '서울특별시 광진구 긴고랑로9길 28 중곡제일시장 103호',
  '음식 종류': '기타 한식',
  '가격 범위': '만원 미만',
  '리뷰개수': 13,
  '평점': '4.0'},
 {'가게이름': '건대우동집',
  '가게 주소': '서울특별시 광진구 아차산로 201 동성빌딩 1F',
  '음식 종류': '기타 한식',
  '가격 범위': '만원 미만',
  '리뷰개수': 9,
  '평점': '4.0'},
 {'가게이름': '은혜떡볶이',
  '가게 주소': '서울특별시 광진구 광나루로 384 2F',
  '음식 종류': '기타 한식',
  '가격 범위': '만원 미만',
  '리뷰개수': 33,
  '평점': '3.9'},
 {'가게이름': '순금이떡볶이',
  '가게 주소': '서울특별시 광진구 자양로43길 43',
  '음식 종류': '기타 한식',
  '가격 범위': '만원

In [12]:
review_list

[{'가게이름': '원조할아버지손두부',
  '리뷰': '예전 초당할머니 순두부는 살짝 변해서여길 못따라옴흰순두부는 맛과 향이 살짝 비릴수 있는데 이집은 전혀 그런거 없이 고소하다.',
  '맛평가': '맛있다'},
 {'가게이름': '원조할아버지손두부',
  '리뷰': '아차산 등산 후 내려오는 길에 필수코스처럼 되어버린 두부집. 늘 대기가 조금씩 있는데 회전이 워낙 빨라 십분 이상 대기한 적이 별로 없는 것 같다. 포장도 가능해서 줄이 길면 그냥 포장해도 된다(포장시 두부에서 물이 빠져 좀 단단해지는 감은 있다) 모두부, 순두부, 콩국수가 있다. 콩국수는 걸쭉한 콩국물이 찐이고 순두부와 모두부 모두 갓 만든 듯한 고소함과 탱글한 질감을 보여준다. 곁들여 나오는 젓갈이 또 별미다. 막걸리 한잔 안할 수가 없는 곳.',
  '맛평가': '맛있다'},
 {'가게이름': '원조할아버지손두부',
  '리뷰': '<다른메뉴 없고 손두부만 파는 집>이 집의 매력포인트 중 하나는 막걸리가 정말 싸다는 것이다. 모든 막걸리 2,500원두부는 부드럽고 탱글탱글 존맛탱반찬도 없고 찐 두부만 합리적인 가격에 내놓는다',
  '맛평가': '맛있다'},
 {'가게이름': '원조할아버지손두부',
  '리뷰': '등산 후 맛보는 기가막힌 #원조할아버지손두부- 직접 만든 유기농 100% 손두부- 저렴한 가격 고퀄리티 맛 ❤️- 웨이팅 있음, 자리순환 빠름 오랜만에 아차산을 등산했어요! 서울 시내 풍경을 눈에 담고 하산 후 그 유명한 할아버지 두부집으로 왔습니다 ! 모두부 /6,000따뜻한 한모가 잘려서 나옵니다. 같이 나오는 새우젓갈과 고추는 두부와 먹기 아주 맛있어요. 젓갈은 비린 향 없이 고추가루양념으로 되어 있는데 호불호 없이 무난하게 사람들이 좋아 할 수 있는 사이드 같아요. 저는 순두부, 모두부 중에 씹는 맛이 느껴지는 모두부를 더 맛있게 먹은거 같아요. 순두부도 맛있었는데 많이 먹지는 못했어요순두부 /4,0001인분 양으로 나오는데 양이 많습니다. 요거트 같아요. 한 입 떠먹

In [13]:
review_frame = pd.DataFrame(review_list)
review_frame

,가게이름,리뷰,맛평가
0,원조할아버지손두부,예전 초당할머니 순두부는 살짝 변해서여길 못따라옴흰순두부는 맛과 향이 살짝 비릴수 ...,맛있다
1,원조할아버지손두부,아차산 등산 후 내려오는 길에 필수코스처럼 되어버린 두부집. 늘 대기가 조금씩 있는...,맛있다
2,원조할아버지손두부,<다른메뉴 없고 손두부만 파는 집>이 집의 매력포인트 중 하나는 막걸리가 정말 싸다...,맛있다
3,원조할아버지손두부,등산 후 맛보는 기가막힌 #원조할아버지손두부- 직접 만든 유기농 100% 손두부- ...,맛있다
4,원조할아버지손두부,드디어 먹어본 아차산 순두부!! 같이 가 준 친구들이 이미 전에도 맛본 경험이 있어...,맛있다
...,...,...,...
6504,리바,비비드아워로 방문.종류는 아주 다양하다거나 신라 정도의 퀄리티는 아니지만 뷰가 예쁘...,괜찮다
6505,리바,"• 딱! 괜찮은 정도에요• 물론, 뷰+분위기는 굉장히 힙하나,• 팥빙수는 정말.. ...",괜찮다
6506,리바,.,괜찮다
6507,몽,분위기 어둑지고 아늑한 곳. 인도풍? 칵테일바 인데 분수도 있고 신발벗고 편히 앉아...,맛있다


In [14]:
detail_frame = pd.DataFrame(detail)
detail_frame

,가게이름,가게 주소,음식 종류,가격 범위,리뷰개수,평점
0,원조할아버지손두부,서울특별시 광진구 자양로 324,한정식 / 백반 / 정통 한식,만원 미만,72,4.5
1,이화만두,서울특별시 광진구 능동로 278-1,한정식 / 백반 / 정통 한식,만원 미만,6,4.3
2,서북면옥,서울특별시 광진구 자양로 199-1,국수 / 면 요리,만원 미만,40,4.2
3,궤도에오르다,서울특별시 광진구 광나루로 416 건희빌딩 1F,기타 한식,만원 미만,40,4.0
4,콩이네,서울특별시 광진구 긴고랑로9길 28 중곡제일시장 103호,기타 한식,만원 미만,13,4.0
...,...,...,...,...,...,...
1152,마우로아,서울특별시 광진구 능동로 320 1F,칵테일 / 와인,3만원-4만원,24,4.2
1153,글래시,서울특별시 광진구 능동로 23-14 1F,칵테일 / 와인,3만원-4만원,19,3.5
1154,리바,서울특별시 광진구 워커힐로 177 비스타 워커힐 1F,칵테일 / 와인,4만원 이상,5,None
1155,몽,서울특별시 광진구 아차산로33길 25,칵테일 / 와인,4만원 이상,1,None


In [15]:
district

'건대_군자_광진_'

In [16]:
review_frame.to_csv('C:/Users/leedo/Documents/ds_study/{}_review_list.csv'.format(district + 'total'), encoding='utf-8-sig')

In [17]:
detail_frame.to_csv('C:/Users/leedo/Documents/ds_study/{}_detail.csv'.format(district + 'total'), encoding='utf-8-sig')